<img src='a3.png'>

[Lösungshinweise](wortsuche.pdf)

#### Beispieldaten (0-5)

In [267]:
nr = 0
f = open('./beispieldaten/worte'+str(nr)+'.txt')
print(f.read())

5 5
4
VOR
RAD
EVA
TORF



Eine mögliche Lösung ist eine Folge von Entscheidungen: 
```
Setze 1.Wort auf Position (0,0) von rechts nach links
Setze 2.Wort auf Position (2,0) nach unten
usw.
```

Wir modellieren einen state als eine Folge solcher Entscheidungen. Die acht möglichen Richtungen kodieren wir im Uhrzeigersinn mit 0-7.
```
0 = Nord, 1 = NordOst, 2 = Ost, usw.
```
Für die obige Situation ergibt sich:
```
state = ((0,0,2),(0,2,4))

```

In [268]:
nr = 0

f = open('./beispieldaten/worte'+str(nr)+'.txt',encoding='utf-8')
rows, cols = [int(k) for k in f.readline().split()]
anz = int(f.readline())
worte = []
for i in range(anz):
    worte.append(f.readline().strip())
worte

['VOR', 'RAD', 'EVA', 'TORF']

In [269]:
startstate = ()   # leeres Tupel

In [270]:
def goaltest(state):
    return len(state) == len(worte)
    

Für die Funktion nextstates übersetzen wir zunächst den übergebenen state in eine Matrix mit den bisher gesetzten Wörtern

In [271]:
import numpy as np
def makeMatrix(state):
    ls = len(state)
    a = np.array(list('.'*rows*cols))
    a = a.reshape(rows,cols)
    
    for k in range(ls):
        i, j, richtung = state[k]
        wort = list(worte[k])
        if richtung == 0:    # Nord
            a[range(i,i-len(wort),-1),j] = wort
        elif richtung == 1:  # NordOst
            a[range(i,i-len(wort),-1),range(j,j+len(wort))] = wort
        elif richtung == 2:  # Ost
            a[i,range(j,j+len(wort))] = wort
        elif richtung == 3:  # SüdOst
            a[range(i,i+len(wort)),range(j,j+len(wort))] = wort
        elif richtung == 4:  # Süd
            a[range(i,i+len(wort)),j] = wort
        elif richtung == 5:  # SüdWest
            a[range(i,i+len(wort)),range(j,j-len(wort),-1)] = wort
        elif richtung == 6:  # West
            a[i,range(j,j-len(wort),-1)] = wort
        elif richtung == 7:  # Nordwest
            a[range(i,i-len(wort),-1),range(j,j-len(wort),-1)] = wort
    return a

In [259]:
state = ((0,0,2),(0,2,4))
makeMatrix(state)

array([['V', 'O', 'R', '.', '.'],
       ['.', '.', 'A', '.', '.'],
       ['.', '.', 'D', '.', '.'],
       ['.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.']], dtype='<U1')

In [272]:
def check(state,wort,i,j,richtung):
    '''
    returned True, wenn im State state das Wort wort an
    die Stelle (i,j) positioniert werden kann 
    und zwar in Richtung richtung.
    '''
    a = makeMatrix(state)
    lw = len(wort)
    leer = list('.'*lw)
    
    if richtung == 0:    # Nord
        ibis = i-lw
        if ibis <= -1: return False
        teil = a[range(i,ibis,-1),j]
    elif richtung == 1:  # NordOst
        ibis = i-lw
        jbis = j+lw
        if ibis <= -1: return False
        if jbis > cols: return False
        teil = a[range(i,ibis,-1),range(j,jbis)]  
    elif richtung == 2:  # Ost
        jbis = j+lw
        if jbis > cols: return False
        teil =  a[i,range(j,jbis)]  
    elif richtung == 3:  # SüdOst
        ibis = i+lw
        jbis = j+lw
        if ibis > rows: return False
        if jbis > cols: return False
        teil = a[range(i,ibis),range(j,jbis)]  
    elif richtung == 4:  # Süd
        ibis = i+lw
        if ibis > rows: return False
        teil = a[range(i,ibis),j] 
    elif richtung == 5:  # SüdWest
        ibis = i+lw
        jbis = j-lw
        if ibis > rows: return False
        if jbis <= -1: return False
        teil = a[range(i,ibis),range(j,jbis,-1)]  
    elif richtung == 6:  # West
        jbis = j-lw
        teil = a[i,range(j,jbis,-1)]  
        if jbis <= -1: return False
    elif richtung == 7:  # Nordwest
        ibis = i-lw
        jbis = j-lw
        teil = a[range(i,ibis,-1),range(j,jbis,-1)]  
        if ibis <= -1: return False
        if jbis <= -1: return False
        
    for k in range(lw):
        if wort[k] != teil[k] and teil[k] != '.':
            return False
    return True

def nextstates(state):
    tmp = []
    wort = worte[len(state)]
    for i in range(rows):
        for j in range(cols):
            for richtung in richtungen:              # die globale Variale richtungen steuert die Schwierigkeit
                if check(state,wort,i,j,richtung):
                    tmp.append(state + ((i,j,richtung),))
    return tmp
        

Es gibt vermutlich viele Lösungen. Alle Lösungen finden sich als Blätter im Suchbaum. Uns reicht eine Lösung. Deswegen verwenden wir die Tiefensuche (dfs). Die nextstates mischen wir, damit immer wieder neue Lösungen gefunden werden.

In [273]:
import random
def dfs(s):
    frontier =  [s]
    prev = {s:None}
    while frontier:
        state = frontier.pop()  
        if goaltest(state):
            return prev, state
        nxt = nextstates(state)
        random.shuffle(nxt)
        for v in nxt:
            if v not in prev:
                frontier.append(v)
                prev[v] = state
    return None 

In [276]:
richtungen = [2,4]
prev, goal = dfs(startstate)

In [277]:
a = makeMatrix(goal)
a

array([['.', 'E', '.', '.', 'T'],
       ['.', 'V', '.', 'R', 'O'],
       ['.', 'A', '.', 'A', 'R'],
       ['.', '.', '.', 'D', 'F'],
       ['V', 'O', 'R', '.', '.']], dtype='<U1')

In [278]:
def fill(a):
    '''
    a: Buchstabenmatrix mit einigen freien Stellen '.'
    returns: None, füllt a an den freien Stellen mit zufälligen Buchstaben
    '''
    start = ord('A')
    for i in range(rows):
        for j in range(cols):
            if a[i,j]=='.':
                a[i,j] = chr(random.randint(start,start+25))
                
def show(a):
    for x in a:
        print(' '.join(x))
fill(a)
show(a)

V E E V T
R V L R O
S A D A R
I T C D F
V O R F O


#### Das vollständige Programm

Beim Beispiel 4 benötigt unser Programm zu lange. Deshalb beschränken wir die zu untersuchenden nextstates auf eine Anzahl, die wir in 
der globalen Variablen *maxVerzweigungen*  mitgeben. Wir riskieren dadurch, dass wir keine Lösung finden. In den Beispielen 0-5 ist das aber nicht der Fall. Damit nicht immer dieselben Stellen und Richtungen untersucht werden, mischen wir in nextstates die Listen, über die wir iterieren.

In [279]:
import numpy as np
import random

def makeMatrix(state):
    ls = len(state)
    a = np.array(list('.'*rows*cols))
    a = a.reshape(rows,cols)
    
    for k in range(ls):
        i, j, richtung = state[k]
        wort = list(worte[k])
        if richtung == 0:    # Nord
            a[range(i,i-len(wort),-1),j] = wort
        elif richtung == 1:  # NordOst
            a[range(i,i-len(wort),-1),range(j,j+len(wort))] = wort
        elif richtung == 2:  # Ost
            a[i,range(j,j+len(wort))] = wort
        elif richtung == 3:  # SüdOst
            a[range(i,i+len(wort)),range(j,j+len(wort))] = wort
        elif richtung == 4:  # Süd
            a[range(i,i+len(wort)),j] = wort
        elif richtung == 5:  # SüdWest
            a[range(i,i+len(wort)),range(j,j-len(wort),-1)] = wort
        elif richtung == 6:  # West
            a[i,range(j,j-len(wort),-1)] = wort
        elif richtung == 7:  # Nordwest
            a[range(i,i-len(wort),-1),range(j,j-len(wort),-1)] = wort
    return a

def check(state,wort,i,j,richtung):
    '''
    returned True, wenn im State state das Wort wort an
    die Stelle (i,j) positioniert werden kann 
    und zwar in Richtung richtung.
    '''
    a = makeMatrix(state)
    lw = len(wort)
    leer = list('.'*lw)
    
    if richtung == 0:    # Nord
        ibis = i-lw
        if ibis <= -1: return False
        teil = a[range(i,ibis,-1),j]
    elif richtung == 1:  # NordOst
        ibis = i-lw
        jbis = j+lw
        if ibis <= -1: return False
        if jbis > cols: return False
        teil = a[range(i,ibis,-1),range(j,jbis)]  
    elif richtung == 2:  # Ost
        jbis = j+lw
        if jbis > cols: return False
        teil =  a[i,range(j,jbis)]  
    elif richtung == 3:  # SüdOst
        ibis = i+lw
        jbis = j+lw
        if ibis > rows: return False
        if jbis > cols: return False
        teil = a[range(i,ibis),range(j,jbis)]  
    elif richtung == 4:  # Süd
        ibis = i+lw
        if ibis > rows: return False
        teil = a[range(i,ibis),j] 
    elif richtung == 5:  # SüdWest
        ibis = i+lw
        jbis = j-lw
        if ibis > rows: return False
        if jbis <= -1: return False
        teil = a[range(i,ibis),range(j,jbis,-1)]  
    elif richtung == 6:  # West
        jbis = j-lw
        teil = a[i,range(j,jbis,-1)]  
        if jbis <= -1: return False
    elif richtung == 7:  # Nordwest
        ibis = i-lw
        jbis = j-lw
        teil = a[range(i,ibis,-1),range(j,jbis,-1)]  
        if ibis <= -1: return False
        if jbis <= -1: return False
        
    for k in range(lw):
        if wort[k] != teil[k] and teil[k] != '.':
            return False
    return True

def fill(a):
    '''
    a: Buchstabenmatrix mit einigen freien Stellen '.'
    returns: None, füllt a an den freien Stellen mit zufälligen Buchstaben
    '''
    start = ord('A')
    for i in range(rows):
        for j in range(cols):
            if a[i,j]=='.':
                a[i,j] = chr(random.randint(start,start+25))
                
def show(a):
    for x in a:
        print(' '.join(x))

def goaltest(state):
    return len(state) == len(worte)

def nextstates(state):
    tmp = []
    wort = worte[len(state)]

    ri = list(range(rows))
    rj = list(range(cols))
    random.shuffle(richtungen)
    random.shuffle(ri)
    random.shuffle(rj)
    for i in ri:
        for j in rj:
            for richtung in richtungen:              # die globale Variale richtungen steuert die Schwierigkeit
                if check(state,wort,i,j,richtung):
                    tmp.append(state + ((i,j,richtung),))
                    if len(tmp) > maxVerzweigungen: return tmp   # die global Variable maxVerzweigungen begrenzt den search-space
    return tmp

def dfs(s):
    frontier =  [s]
    while frontier:
        state = frontier.pop()  
        if goaltest(state):
            return state                 # wir geben nur einen goalstate zurück
        for v in nextstates(state):
            frontier.append(v)


In [282]:
%%time
nr = 4
schwierigkeit = 3

maxVerzweigungen = 5
f = open('./beispieldaten/worte'+str(nr)+'.txt',encoding='utf-8')
rows, cols = [int(k) for k in f.readline().split()]
anz = int(f.readline())
worte = []
for i in range(anz):
    worte.append(f.readline().strip())

print(worte)
print()


if schwierigkeit == 1:
    richtungen = [2,4]
elif schwierigkeit == 2:
    richtungen = [0,2,4,6]
elif schwierigkeit == 3:
    richtungen = [0,1,2,3,4,5,6,7]
goal = dfs(startstate)
a = makeMatrix(goal)  
show(a)
print()
fill(a)
show(a)
 
 

['ABRUFDATUM', 'ABSATZ', 'ACHTUNG', 'ALLMUSIC', 'ARCHIV', 'ARCHIVBOT', 'ARCHIVIERUNG', 'AUS', 'AUT', 'AUTOARCHIV', 'BABEL', 'BAUSTELLE', 'BBKL', 'BEGRIFFSKLÄRUNG', 'BEGRIFFSKLÄRUNGSHINWEIS', 'BEL', 'BELEGE', 'BENUTZER', 'BGR', 'BIBRECORD', 'BOOLAND', 'CAN', 'CENTER', 'CHARTS', 'COL', 'COMMONS', 'COMMONSCAT', 'COORDINATE', 'COORDINATEMAP', 'DDB', 'DEU', 'DISKUSSIONSSEITE', 'DOI', 'ERLEDIGT', 'FARBLEGENDE', 'FILM', 'FN', 'FNZ', 'FOLGENLEISTE', 'FRA', 'FUSSBALLDATEN', 'GEOQUELLE', 'GER', 'GNIS', 'HÖHE', 'IMDB', 'INFO', 'INFORMATION', 'INTERNETQUELLE', 'IPA', 'KALENDERSTIL', 'KASTEN', 'KATEGORIEGRAPH', 'LANG', 'LITERATUR', 'LIZENZUMSTELLUNG', 'MEDAILLENSPIEGEL', 'MULTILINGUAL', 'MUSIKCHARTS', 'NAVFRAME', 'NAVIGATIONSLEISTE', 'NOCOMMONS', 'ÖSTERREICHBEZOGEN', 'PERSONENLEISTE', 'PING', 'POSITIONSKARTE', 'PRO', 'SMILEY', 'SORT', 'TAXOBOX', 'TEXT', 'WAPPENRECHT', 'WEBARCHIV', 'WIKIDATA', 'WIKISOURCE', 'WIKTIONARY', 'ZITATION']

. . . G N I P . N E G O Z E B H C I E R R E T S Ö . . . . . . .
. 

#### Dokumentation einiger Ergebnisse

worte4 - Schwierigkeit 3 - maxVerzweigung = 10
```
. . . . . . . . . . . . . . . A R C H I V B O T . . . . . . . .
. L . . . . . . . . . . . . . B . . . . . . A B R U F D A T U M
. A . . . . . . S . . . . . . D . . . . L I T S R E D N E L A K
. U . . . . . . I . . . . . Z M . . . . . . . . . . . . . . . .
. G R . . . . . E . . . . . I I . . . . . . . . . . . D . . . .
. N E I . . E . W . D . . . T . . . T . L E B A B . . N . . . .
. I Z N . . T . N O . A B S A T Z W . E . . . . . . . A . . . .
. L T T . . S . I G W . . . T . . . E . X E D N E G E L B R A F
. I U E . . I . H N I E . . I . . R . B T T . . . . . O . . . .
. T N R . . E M S I K . T . O . . . U S A A R C H I V O . . . .
. L E N . . L . G S T . . S N . N . I T . R . . S . . B . . . .
C U B E . I S . N . I . . . I . F E . . A . C O . . . . . . . .
O M N T F . N . U . O . . A . E L . T . . R R H . . O . . . . .
O . A Q . . O . R M N . E . T N L G . . . T E . I . R . . . . .
R . V U . . I . Ä U A G . T E A I N . . . . . T . V P . V . . .
D . F E . . T . L S R . N G R D D . E . . . . . I . . I S . . .
I . R L . . A . K I Y . L U E A . I . N . . . . . L H N N . . .
N . A L . . G . S K . O . L R . K A K . O . . . . C . E O . . .
A . M E . . I . F C F G R . . Ä . S U I . S . . R . . T M . . .
T . E . . . V . F H . E N . . . L . N T W . R A . . . S M . . .
E L R . . . A . I A . . . U . . . K . O . . O E . . C A O . . B
M . I E . . N . R R . . . . R . W . S . I T . . P . I K C . . E
A . . Z G . X . G T . E . . . E . I . F U T . . . . S . . . I L
P . . . E O Ö . E S . . T C . . I . K A F . I . . . U . . . N .
. . . . B N . S B . . . O A . . L V . I . I . S . . M . B . F .
. . . O . . Z . T . . M . F N E . . I K S . R . O . L . . G O .
. . X S . D . U . E M . U . G I Y . A H . O . G . P L . . . R .
. A . N . F R . M O R S . E . E D T . . C . U . E . A . . G M .
T . . O . R . O N S S R I . L . E R . . . R . R A B . . . E A D
T . . M . A . S C B T P E I . G L . O . . . A . C Z . . . O T E
H . . M . . C . A E S E M I O . K . . O . . . . H E N . . Q I U
C . . O . A . L . N R S L R C . B . . C C . D . T . . F . U O .
E . . C T . L . E C . B I L R H B . O . A D . . U . . . . E N .
R . . O . D . L H A . E I . U E B L . . B N . . N . . G H L . .
N . . N A . L A U . G . . B . N T E . . . E . O G . N Ö . L . .
E . . T . I R S . R . . . . . . G N Z . . G . F . I H A . E . .
P . E . A T . L A N G . . . . . . . E O . E . N P E P . . . . .
P N . D S . . P E L L E T S U A B . . C G L . I . I . . . . . .
A . E . . . H . . . . . . . . . . . . . . E . . . . . . . . . .
W M D I S K U S S I O N S S E I T E . . . B N . . . . . . . . .

I Y R H T P T L F E R T X E I A R C H I V B O T U B P M T C X P
S L F M R A U V U D R E E I R B R U V I B A A B R U F D A T U M
I A E D X S F N S S J U M V Y D U U M G L I T S R E D N E L A K
O U H E D Y H W I N C K V D Z M N U L G F T U J P E J R Y H O M
P G R O G S A E E Q M Y K K I I D Z R E V Z E Y M R M D U H A Q
W N E I V U E Q W J D Z T S T F N H T Z L E B A B F H N B L Q U
P I Z N I Q T Y N O X A B S A T Z W T E F U P Y U R C A Q Y D S
J L T T G F S E I G W U Q I T D Z J E N X E D N E G E L B R A F
M I U E R K I T H N I E P V I G X R A B T T T R O O Z O P U H D
C T N R S Z E M S I K G T N O U M I U S A A R C H I V O Y R B V
D L E N X W L G G S T A R S N K N A I T R R O I S L A B P W L T
C U B E M I S K N J I U U Z I S F E P C A U C O Y G T A O U N B
O M N T F W N X U V O A K A R E L B T R W R R H Y N O H I J N B
O B A Q G E O Q R M N C E M T N L G O P E T E K I E R Y C F N S
R D V U U I I J Ä U A G N T E A I N L G F Z K T B V P T V I W N
D S F E Z D T H L S R R N G R D D S E Y I B T G I H W I S Y T T
I O R L T F A M K I Y J L U E A T I Q N E W G M F L H N N L I Q
N B A L V P G V S K L O H L R R K A K U O M J B C C D E O O S L
A T M E Y X I Q F C F G R P X Ä T S U I U S X X R R A T M M T C
T T E J P P V D F H N E N O Q L L V N T W B R A B F W S M R C W
E L R V Q R A F I A G U H U C B C K D O C E O E N Z C A O B W B
M T I E W Q N I R R O D C A R S W N S G I T G N P U I K C F X E
A A S Z G J X Y G T Z E R N R E E I Y F U T C Q Y O S I E H I L
P Z V W E O Ö X E S M M T C C F I R K A F E I D U C U O J F N S
O C Y F B N L S B W K U O A P U L V K I P I S S X W M L B X F T
V Q M O Y W Z W T R K M W F N E V Z I K S I R E O A L U W G O G
F P X S X D P U J E M N U X G I Y B A H E O O G L P L N I G R C
Z A N N B F R T M O R S T E O E D T D V C P U N E F A B J G M Y
T C I O T R Z O N S S R I O L O E R J A E R E R A B E F G E A D
T Z V M K A M S C B T P E I P G L I O R H K A V C Z B Q A O T E
H V G M T N C V A E S E M I O J K Q W O Q K K P H E N B L Q I U
C Z L O A A N L F N R S L R C X B E G C C O D D T J I F Z U O R
E X B C T L L Y E C W B I L R H B P O H A D S Z U A H N I E N T
R Q T O E D K L H A B E I W U E B L V E B N Y Y N B V G H L W Q
N O D N A P L A U Z G X C B A N T E D Q T E H O G N N Ö T L Y H
E Y J T C I R S K R Z N F G B O G N Z I X G X F B I H A W E L U
P R E F A T Q L A N G G I R J H F J E O A E U N P E P F E J P J
P N X D S Q E P E L L E T S U A B Q T C G L R I N I Q Q Y V T N
A C E S W W H A E G Q M C P O B E C Z F B E A X F N L R U Y Q S
W M D I S K U S S I O N S S E I T E C W I B N G F N M L V X Q A
CPU times: total: 13.7 s
Wall time: 13.7 s
```

Hinweis: Dieses Problem lässt sich mit Backtracking besser lösen.